In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
data_dir = 'DATA'

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
num_classes = len(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 350 files belonging to 7 classes.
Using 280 files for training.
Found 350 files belonging to 7 classes.
Using 70 files for validation.


In [15]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

epochs=3500
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

model.save('mi_modelo_entrenado')

Epoch 1/3500
9/9 [==============================] - 1s 28ms/step - loss: 2.1970 - accuracy: 0.1286 - val_loss: 1.9572 - val_accuracy: 0.1000
Epoch 2/3500
9/9 [==============================] - 0s 11ms/step - loss: 1.9535 - accuracy: 0.1607 - val_loss: 1.9704 - val_accuracy: 0.1143
Epoch 3/3500
9/9 [==============================] - 0s 11ms/step - loss: 1.9229 - accuracy: 0.2464 - val_loss: 1.9725 - val_accuracy: 0.1286
Epoch 4/3500
9/9 [==============================] - 0s 11ms/step - loss: 1.8654 - accuracy: 0.3357 - val_loss: 1.9466 - val_accuracy: 0.1571
Epoch 5/3500
9/9 [==============================] - 0s 11ms/step - loss: 1.7349 - accuracy: 0.3929 - val_loss: 1.9499 - val_accuracy: 0.1714
Epoch 6/3500
9/9 [==============================] - 0s 11ms/step - loss: 1.5713 - accuracy: 0.4393 - val_loss: 1.9723 - val_accuracy: 0.1429
Epoch 7/3500
9/9 [==============================] - 0s 11ms/step - loss: 1.2913 - accuracy: 0.5464 - val_loss: 3.2469 - val_accuracy: 0.1000
Epoch 8/3500


INFO:tensorflow:Assets written to: mi_modelo_entrenado\assets


INFO:tensorflow:Assets written to: mi_modelo_entrenado\assets


In [13]:
import cv2
import numpy as np

In [14]:
model = tf.keras.models.load_model('mi_modelo_entrenado')
class_names = ['Angry', 'Disguist', 'Fear','Happy','Neutral', 'Sad', 'Surprise']

cap = cv2.VideoCapture(0)

def preprocess(frame, model):
    target_size = (model.input_shape[1], model.input_shape[2])
    frame = cv2.resize(frame, target_size)
    frame = frame / 255.0
    return frame

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_processed = preprocess(frame, model)
    prediction = model.predict(np.expand_dims(frame_processed, axis=0))
    probabilities = tf.nn.softmax(prediction[0])
    predicted_class = np.argmax(probabilities)
    confidence = np.max(probabilities) * 100
    emotion = class_names[predicted_class]

    cv2.putText(frame, f'{emotion}: {confidence:.2f}%', (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('Emotion Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 10ms/step
